In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

In [2]:
# define parameters 

ticker = 'WMT'
startDate = '2022-11-01'
endDate = '2022-12-31'
dataInterval = '5m'

fastSMA = 20 #9
normalSMA = 100 #50
slowSMA = 200 #200

# moving average period
rollingPeriod = 21

# entry ratio
entryPercentile = 0.10

# exit ratio
exitPercentile = 0.90

In [3]:
# get data
df = yf.download(ticker, startDate, endDate, interval = dataInterval)
df.reset_index(inplace=True)

df['Date'] = df['Datetime'].dt.date

[*********************100%***********************]  1 of 1 completed


In [30]:
df_sample = df[df['Date']==datetime.date(2022, 11, 1)]
df_sample.reset_index(inplace=True)

In [41]:
min_Low = []
min_Close = []
max_High = []
max_Close = []
for i in range(1,len(df_sample)):
    min_Close.append(df_sample['Datetime'][df_sample[:i]['Close'].idxmin()])
    max_Close.append(df_sample['Datetime'][df_sample[:i]['Close'].idxmax()])
    min_Low.append(df_sample['Datetime'][df_sample[:i]['Low'].idxmin()])
    max_High.append(df_sample['Datetime'][df_sample[:i]['High'].idxmax()])

In [44]:
data = {'ticker': ['WMT'] * len(min_Low),
        'Date': df_sample['Datetime'][1:],
        'dataInterval': ['5m'] * len(min_Low),
        'min_Low': min_Low,
        'min_Close': min_Close,
        'max_High': max_High,
        'max_Close': max_Close
       }
pd.DataFrame(data)

,ticker,Date,dataInterval,min_Low,min_Close,max_High,max_Close
1,WMT,2022-11-01 09:35:00-04:00,5m,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
2,WMT,2022-11-01 09:40:00-04:00,5m,2022-11-01 09:30:00-04:00,2022-11-01 09:35:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
3,WMT,2022-11-01 09:45:00-04:00,5m,2022-11-01 09:30:00-04:00,2022-11-01 09:40:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
4,WMT,2022-11-01 09:50:00-04:00,5m,2022-11-01 09:45:00-04:00,2022-11-01 09:45:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
5,WMT,2022-11-01 09:55:00-04:00,5m,2022-11-01 09:45:00-04:00,2022-11-01 09:50:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
...,...,...,...,...,...,...,...
73,WMT,2022-11-01 15:35:00-04:00,5m,2022-11-01 11:00:00-04:00,2022-11-01 11:05:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
74,WMT,2022-11-01 15:40:00-04:00,5m,2022-11-01 11:00:00-04:00,2022-11-01 11:05:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
75,WMT,2022-11-01 15:45:00-04:00,5m,2022-11-01 11:00:00-04:00,2022-11-01 11:05:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00
76,WMT,2022-11-01 15:50:00-04:00,5m,2022-11-01 11:00:00-04:00,2022-11-01 11:05:00-04:00,2022-11-01 09:30:00-04:00,2022-11-01 09:30:00-04:00


In [43]:
df_sample

,index,Datetime,Open,High,Low,Close,Adj Close,Volume,Date
0,0,2022-11-01 09:30:00-04:00,142.970001,143.550003,142.839996,143.350006,143.350006,200605,2022-11-01
1,1,2022-11-01 09:35:00-04:00,143.419998,143.419998,142.984604,143.184998,143.184998,66872,2022-11-01
2,2,2022-11-01 09:40:00-04:00,143.179993,143.399994,143.029999,143.080002,143.080002,72084,2022-11-01
3,3,2022-11-01 09:45:00-04:00,143.000000,143.000000,142.294998,142.470001,142.470001,60058,2022-11-01
4,4,2022-11-01 09:50:00-04:00,142.509995,142.720001,142.399994,142.427704,142.427704,95033,2022-11-01
...,...,...,...,...,...,...,...,...,...
73,73,2022-11-01 15:35:00-04:00,141.720001,141.910004,141.720001,141.823502,141.823502,53360,2022-11-01
74,74,2022-11-01 15:40:00-04:00,141.809998,141.899994,141.690002,141.800003,141.800003,54106,2022-11-01
75,75,2022-11-01 15:45:00-04:00,141.809998,141.949997,141.789993,141.949997,141.949997,74301,2022-11-01
76,76,2022-11-01 15:50:00-04:00,141.949997,142.000000,141.679993,141.729996,141.729996,125954,2022-11-01
